In [67]:
from maplib import Mapping
import polars as pl
from math import floor

# Templates

In [68]:
instance_mapping = """
@prefix tpl:<https://github.com/magbak/chrontext/templates#>.
@prefix rds:<https://github.com/magbak/chrontext/rds_power#>.
@prefix ct:<https://github.com/magbak/chrontext#>.
@prefix rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix rdfs:<http://www.w3.org/2000/01/rdf-schema#>.

tpl:Site [?SiteIRI, ?SiteName] :: {
    ottr:Triple(?SiteIRI, rdfs:label, ?SiteName),
    ottr:Triple(?SiteIRI, rdf:type, rds:Site)
    } .
    
tpl:FunctionalAspect [?SourceIRI, ?TargetIRI, ?TargetAspectNodeIRI, ?Label] :: {
    ottr:Triple(?SourceIRI, rds:hasFunctionalAspect, ?TargetAspectNodeIRI),
    ottr:Triple(?TargetIRI, rds:hasFunctionalAspectNode, ?TargetAspectNodeIRI),
    ottr:Triple(?TargetAspectNodeIRI, rdfs:label, ?Label)
} .

tpl:RDSSystem [?SystemIRI, xsd:anyURI ?RDSType, ?Label] :: {
    ottr:Triple(?SystemIRI, rdf:type, ?RDSType),
    ottr:Triple(?SystemIRI, rdfs:label, ?Label)
} .

tpl:StaticProperty [?ParentIRI, ?ValueNodeIRI, ?Label, ?Value] :: {
    ottr:Triple(?ParentIRI, ct:hasStaticProperty, ?ValueNodeIRI),
    ottr:Triple(?ValueNodeIRI, rdfs:label, ?Label),
    ottr:Triple(?ValueNodeIRI, ct:hasStaticValue, ?Value)
} .

tpl:Timeseries [?ParentIRI, ?TimeseriesNodeIRI, ?Label, ?ExternalId, xsd:anyURI ?Datatype] :: {
    ottr:Triple(?ParentIRI, ct:hasTimeseries, ?TimeseriesNodeIRI),
    ottr:Triple(?TimeseriesNodeIRI, ct:hasExternalId, ?ExternalId),
    ottr:Triple(?TimeseriesNodeIRI, ct:hasDatatype, ?Datatype),
    ottr:Triple(?TimeseriesNodeIRI, rdfs:label, ?Label)
} .
"""

In [69]:
n = 400

# Mapping object

In [70]:
mapping = Mapping([instance_mapping])

In [71]:
# Used as a prefix
wpex = "https://github.com/magbak/chrontext/windpower_example#"
rds = "https://github.com/magbak/chrontext/rds_power#"

In [72]:
site_iris = [wpex + "Site" + str(i) for i in range(4)]
sites = pl.DataFrame({"SiteName": ["Wind Mountain", "Gale Valley", "Gusty Plains", "Breezy Field"],
                      "SiteIRI": site_iris})
mapping.expand("tpl:Site", sites)

# Wind turbines

In [73]:
wind_turbine_iris = [wpex + "WindTurbine" + str(i) for i in range (1, n+1)]
wind_turbines = pl.DataFrame(
    {
    "RDSType": [rds + "A"]*n,
    "Label": ["Wind turbine " + str(i) for i in range(1,n+1)],
    "SystemIRI": wind_turbine_iris
    })

mapping.expand("tpl:RDSSystem", wind_turbines)

In [74]:
operating_external_ids = ["oper" + str(i) for i in range(1,n+1)]
operating = pl.DataFrame ({
    "Label": ["Operating"]*n,
    "ParentIRI" :wind_turbine_iris,
    "ExternalId": operating_external_ids,
    "Datatype": ["http://www.w3.org/2001/XMLSchema#boolean"]*n
})
operating = operating.with_column(pl.Series("TimeseriesNodeIRI", [wpex]*operating.height) + operating["ExternalId"])
mapping.expand("tpl:Timeseries", operating)

In [75]:
maximum_power_values = [[5_000_000,10_000_000,15_000_000][i%3] for i in range(1,n+1)]
maximum_power_node_iri = [wpex + "WindTurbineMaximumPower" + str(i) for i in range(1, n+1)]
maximum_power = pl.DataFrame ({
    "Label": ["MaximumPower"] * n,
    "Value": maximum_power_values,
    "ParentIRI": wind_turbine_iris,
    "ValueNodeIRI": maximum_power_node_iri
})
mapping.expand("tpl:StaticProperty", 
               df=maximum_power)

In [76]:
def add_aspect_labeling_by_source(df:pl.DataFrame, prefix:str) -> pl.DataFrame:
   label_df = df.groupby("SourceIRI", maintain_order=True).apply(lambda x:pl.DataFrame({"Label":[prefix + str(i) for i in range(1,x.height+1)]}))
   df = df.with_column(label_df["Label"])
   return df

In [77]:
target_aspect_node_iri = [wpex + "WindTurbineFunctionalAspect" + str(i) for i in range(1, n+1)]

site_has_wind_turbine = pl.DataFrame({
    "SourceIRI":[site_iris[floor(i/(n/sites.height))] for i in range(0,n)],
    "TargetIRI":wind_turbine_iris,
    "TargetAspectNodeIRI": target_aspect_node_iri
})
site_has_wind_turbine = add_aspect_labeling_by_source(site_has_wind_turbine, "A")
mapping.expand("tpl:FunctionalAspect", site_has_wind_turbine)

# Generator systems

In [78]:
generator_system_iris = [wpex + "GeneratorSystem" + str(i) for i in range(1,n+1)]
generator_systems = pl.DataFrame(
    {
    "SystemIRI":generator_system_iris,
    "Label": ["Generator system"]*n,
    "RDSType": [rds + "RA"]*n
    })
mapping.expand("tpl:RDSSystem", generator_systems)

In [79]:
wind_turbine_has_generator_system = pl.DataFrame ({
    "SourceIRI": wind_turbine_iris,
    "TargetIRI": generator_system_iris,
    "TargetAspectNodeIRI": [wpex + "GeneratorSystemFunctionalAspect" + str(i) for i in range(1, n+1)]
})
wind_turbine_has_generator_system = add_aspect_labeling_by_source(wind_turbine_has_generator_system, "RA")
mapping.expand("tpl:FunctionalAspect", wind_turbine_has_generator_system)

# Generator

In [80]:
generator_iris = [wpex + "Generator" + str(i) for i in range(1,n+1)]
generators = pl.DataFrame(
    {
    "SystemIRI": generator_iris,
    "RDSType": [rds + "GAA"]*n,
    "Label" : ["Generator"]*n
    })
mapping.expand("tpl:RDSSystem", generators)

In [81]:
energy_production_external_ids = ["ep" + str(i) for i in range(1,n+1)]
energy_production = pl.DataFrame ({
    "Label": ["Production"]*n,
    "ParentIRI" :generator_iris,
    "ExternalId": energy_production_external_ids,
    "Datatype": ["http://www.w3.org/2001/XMLSchema#double"]*n
})
energy_production = energy_production.with_column(pl.Series("TimeseriesNodeIRI", [wpex]*energy_production.height)  + energy_production["ExternalId"])
mapping.expand("tpl:Timeseries", energy_production)

In [82]:
generator_system_has_generator = pl.DataFrame ({
    "SourceIRI": generator_system_iris,
    "TargetIRI": generator_iris,
    "TargetAspectNodeIRI": [wpex + "GeneratorFunctionalAspect" + str(i) for i in range(1, n+1)]
})
generator_system_has_generator = add_aspect_labeling_by_source(generator_system_has_generator, "GAA")

mapping.expand("tpl:FunctionalAspect", generator_system_has_generator)

# Weather measuring system

In [83]:
wms_iris = [wpex + "WeatherMeasuringSystem" + str(i) for i in range(1,n+1)]
wms = pl.DataFrame(
    {
    "SystemIRI":wms_iris,
    "RDSType": [rds + "LE"]*n,
    "Label": ["Weather Measuring System"]*n
    })
mapping.expand("tpl:RDSSystem", wms)

In [84]:
wind_speed_external_ids = ["wsp" + str(i) for i in range(1,n+1)]
wind_speed = pl.DataFrame ({
    "Label": ["Windspeed"]*n,
    "ParentIRI" :wms_iris,
    "ExternalId": wind_speed_external_ids,
    "Datatype": ["http://www.w3.org/2001/XMLSchema#double"]*n
})
wind_speed = wind_speed.with_column(pl.Series("TimeseriesNodeIRI", [wpex]*wind_speed.height)  + wind_speed["ExternalId"])

mapping.expand("tpl:Timeseries", wind_speed)

In [85]:
wind_direction_external_ids = ["wdir" + str(i) for i in range(1,n+1)]
wind_direction = pl.DataFrame ({
    "Label": ["WindDirection"]*n,
    "ParentIRI" :wms_iris,
    "ExternalId": wind_direction_external_ids,
    "Datatype": ["http://www.w3.org/2001/XMLSchema#double"]*n
})
wind_direction = wind_direction.with_column(pl.Series("TimeseriesNodeIRI", [wpex]*wind_direction.height)  + wind_direction["ExternalId"])

mapping.expand("tpl:Timeseries", wind_direction)

In [86]:
wind_turbine_has_wms = pl.DataFrame ({
    "SourceIRI": wind_turbine_iris,
    "TargetIRI": wms_iris,
    "TargetAspectNodeIRI": [wpex + "WMSFunctionalAspect" + str(i) for i in range(1, n+1)]
})
wind_turbine_has_wms = add_aspect_labeling_by_source(wind_turbine_has_wms, "LE")

mapping.expand("tpl:FunctionalAspect", 
               df=wind_turbine_has_wms)

In [87]:
mapping.write_ntriples("windpower.nt")

In [88]:
df_external_ids = pl.DataFrame({
    "MaximumPowerValue": maximum_power_values,
    "Operating":operating_external_ids,
    "EnergyProduction":energy_production_external_ids,
    "WindSpeed":wind_speed_external_ids,
    "WindDirection":wind_direction_external_ids
})
df_external_ids.write_csv("external_ids.csv")